## Arabic:

### 1. load dataset:

In [1]:
from tools.readers.CT20_AR_reader import readTweets, readLabels, readTopics

In [2]:
# Read in tweets, labels & topics using helper functions
tweets = readTweets()
labels = readLabels()
topics = readTopics()

In [3]:
import emoji
tweets_df = labels
tweets_df['full_text_ar'] = 0

for tweet in tweets:
    # For each tweet ID, find row with that ID in DataFrame and add Arabic text to that row
    tweet_idx = tweets_df[tweets_df['tweetID'] == tweet['id']].index[0]
    tweets_df.iloc[tweet_idx, tweets_df.columns.get_loc('full_text_ar')] = tweet['full_text']

In [4]:
tweets_df.loc[:5]

,topicID,tweetID,label,full_text_ar
0,CT20-AR-05,1221585936095555584,0,بين ثورة لبنان ال١٠٠ يوم وبين ٤٠ العراق \nتتر ورُعاع ايران ما زالت رائحة البارود والنار تفوح منهم ، من حرق الخيم وإطلاق الرصاص على المتظاهرين \nوأكتافهم تعبت من بناء حواجز بينهم وبين المواطنين لحماية مملكتهم الهشة \n\nرُعاع القرن الحالي \n#ايران #العراق #لبنان \n#لبنان_النا_مش_الكن
1,CT20-AR-05,1221587916750753793,0,#لبنان_النا_مش_الكن\nأرملة فقيرة\nعليهاإيقاف خدمات\nخلونانفرحهاوعيالها\nليس لهم بعد الله سبحانه إلاأنتم\nبيت أجاروعايشه ع الضمان\nوفاتورة الكهرباء أهم وأقل شي سداد نص الفاتورة\nوالفصل بعديوم\nأسأل الله يفرج همهم\nوالله مبلغهم مو معجزة\nالتنفيذ؛2030633080(77ألف)\nالكهرباء؛10097517557(7آلاف) https://t.co/1kzQxf2XeB
2,CT20-AR-05,1221602993918894081,0,#جدار_العار\nالحيتان الحيطان..عمروا بينن و بين الشعب..حيطان
3,CT20-AR-05,1221653116036304896,0,هل يحمي #جدار_العار السلطة السياسية الفاسدة من شعبها الثائر ضدها؟ https://t.co/HYQnmpI07L
4,CT20-AR-05,1221663019345874944,1,موازنة شو؟!\n١- ارقام غير صحيحة.\n٢- حكومة لم تنل الثقة بعد، يعني غير مسؤولة عن ارقام وضعتها الحكومة السابقة.\nانّو جلسة استفزاز للشعب؟!\nاذا بدكن الناس تعطيكم فرصة، بالقليلة كونوا منطقيين!\n#لبنان_ينتفض
5,CT20-AR-05,1221688210834759680,0,#سندريلات_مرهج :تزامناً مع انعقاد جلسة #الموازنة النيابية اليوم الاثنين، تابعونا (إن شئتم) عبر شاشة NBN الساعة ١٠ ونصف صباحا في حديث سياسي مباشر حول المستجدات السياسية على الساحتين المحلية والدولية بضيافة الإعلامية أمل فضول 🙋‍♀️\n@nbntweets https://t.co/Tk5frOg2gh


### 2. Arabic text pre-processing: (using Farasa api)

>1. remove url.
2. diacritization.
3. tokenisation.
4. lemmanisation.
5. remove stop words.
6. remove punctuation.
7. remove emoji.

In [5]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [6]:
import re
url_pattern = r"https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}"

In [21]:
# using Farasa API.
from nltk.corpus import stopwords

def tokenizer(text):
    import http.client # import http.client in function to avoid connected time out.
    import json
    text = text.replace('\n', ' ').replace('"', '') # otherwise JSONDecodeError.
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    payload = "{\"text\":\"%s\"}"% text
    payload = payload.encode('utf-8')
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    conn.request("POST", "/msa/webapi/segmenter", payload, headers)
    res = conn.getresponse()
    data = res.read().decode('utf-8')
    data_dict = json.loads(data)
    return data_dict['segtext']

In [22]:
def lemmatizer(text):
    import http.client
    import json
    text = text.replace('\n', ' ').replace('"', '') 
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    payload = "{\"text\":\"%s\"}"% text
    payload = payload.encode('utf-8')
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    conn.request("POST", "/msa/webapi/lemma", payload, headers)
    res = conn.getresponse()
    data = res.read().decode('utf-8')
    data_dict = json.loads(data)
    return data_dict['result']

In [23]:
def diacritizer(text):
    import http.client
    import json
    text = text.replace('\n', ' ').replace('"', '') 
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    payload = "{\"text\":\"%s\"}"% text
    payload = payload.encode('utf-8')
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    conn.request("POST", "/msa/webapi/diacritizeV2", payload, headers)
    res = conn.getresponse()
    data = res.read().decode('utf-8')
    data_dict = json.loads(data)
    return data_dict['output'] # return string.

In [24]:
def list_to_str(l): # Farasa api accept string as input.
    text = str()
    for w in l:
        text+= w
        text+=' '
    return text

In [25]:
def text_processor(df):
    for i in range(len(df)):
        # 1. remove url.
        text = re.sub(url_pattern, "", df.loc[i, 'full_text_ar'])
        # 2. diacritization.
        diac_text = diacritizer(text)
        # 3. tokenisation.
        token_word = tokenizer(diac_text)
        text = list_to_str(token_word)
        # 4. Lemmatizer.
        token_word = lemmatizer(text)
        # 5. remove stop words.
        cleaned_words = [word for word in token_word if word not in stopwords.words("arabic")]
        # 6. remove punctuation.
        characters = [',','’', '\'','.','DBSCAN', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%','-','...','^','{','}']
        words_lists = [word for word in cleaned_words if word not in characters]
        text = list_to_str(words_lists)
        # 7. remove emoji
        df.loc[i, 'full_text_ar'] = remove_emoji(text)
    return df

In [26]:
df = text_processor(tweets_df) # 20 minutes

In [27]:
df.to_csv("processed_Arabic_train.csv")

In [28]:
df.loc[:3]

,topicID,tweetID,label,full_text_ar
0,CT20-AR-05,1221585936095555584,0,ثور ة لبنان ال يوم و ال عراق تتر و رعاع ايران زال ت رائح ة ال بارود و ال نار فاح ، حرق ال خيم و إطلاق ال رصاص ال متظاهر ين و كتف تعب ت بناء حاجز و ال مواطن ين ل حماي ة مملك ت ال هش ة رعاع ال قرن ال حال #ايران #العراق #لبنان #لبنان_النا_مش_الكن
1,CT20-AR-05,1221587916750753793,0,#لبنان_النا_مش_الكن أرمل ة فقير ة عليهاإيقاف خدم ات خلونانفرحهاوعيالها ل الله سبحان ه إلاأنتم بيت أجاروعايشه ع ال ضمان و فاتور ة ال كهرباء أهم و شي سداد نص ال فاتور ة و ال فصل بعديوم سأل الله أفرج و الله مبلغ مو معجز ة ال تنفيذ ؛ 2030633080 77 ألف ال كهرباء ؛ 10097517557 7 ألف
2,CT20-AR-05,1221602993918894081,0,#جدار_العار ال حو ت ال حيطان عمر وا ن و ال شعب حيطان
3,CT20-AR-05,1221653116036304896,0,حمى #جدار_العار ال سلط ة ال سياسي ة ال فاسد ة شعب ال ثائر ضد ؟


### 3. Aravec:

download pre-trained word embedding: https://github.com/bakrianoo/aravec

In [29]:
import gensim
import re
import numpy as np
from nltk import ngrams

from utilities import * # import utilities.py module

In [34]:
t_model = gensim.models.Word2Vec.load('tweet_cbow_300/tweets_cbow_300')

In [37]:
word_vector = t_model.wv['ال']

In [39]:
word_vector.shape

(300,)

In [40]:
train_mean_X = list()

for i in range(len(df)):
    train_text = [t_model.wv[x] for x in df.loc[i, 'full_text_ar'].split(' ') if x in t_model]
    vector = np.array(train_text).mean(axis=0)
    train_mean_X.append(vector)
     
train_mean_X = np.array(train_mean_X)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.


In [42]:
print(df.shape)

print(train_mean_X.shape)

(1500, 4)
(1500, 300)


In [43]:
from sklearn.linear_model import LogisticRegression

train_y = np.array(df['label'])

In [44]:
train_y.shape

(1500,)

In [45]:
from sklearn.model_selection import train_test_split
 
train_mean_X, dev_mean_X, train_y, test_y = train_test_split(train_mean_X, train_y,test_size=0.3,random_state=0)

In [46]:
print(train_mean_X.shape)
print(dev_mean_X.shape)
print(train_y.shape)
print(test_y.shape)

(1050, 300)
(450, 300)
(1050,)
(450,)


In [47]:
clf_mean = LogisticRegression(random_state=0, solver='lbfgs').fit(train_mean_X, train_y)
print("mean score: ", clf_mean.score(dev_mean_X, test_y))

mean score:  0.7666666666666667


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
